# Create undersampled k-space
This demonstration shows how to create different undersampled k-space data which can be used either directly for image reconstruction or used to simulate MR data acquisition of a new object.

This demo is a 'script', i.e. intended to be run step by step in a
Python notebook such as Jupyter. It is organised in 'cells'. Jupyter displays these
cells nicely and allows you to run each cell on its own.

First version: 18th of June 2021
Author: Christoph Kolbitsch

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2021 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2021 University College London.  
Copyright 2015 - 2021 Physikalisch-Technische Bundesanstalt.

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging 
(http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In the previous MR notebooks such as `e_advanced_recon` we used an existing MR raw data file with regular Cartesian sampling and an undersampling factor (R) of 4. If we would like to repeate the notebook for a dataset with R =  6, we would need to go to the MR scanner and acquire new raw data. This of course is one of the strengths of __SIRF__ because it means all our developed code can be applied to real data in a straight forward way, but sometimes it would be nice to switch from R = 4 to R = 6 by adapting a parameter in our script. Or go from regular Cartesian undersampling to random Cartesian undersampling without having to implement a new trajectory on the MR scanner.

This notebook will show how we can achieve this at least to a certain degree. The idea is to start with a fully sampled Cartesian data set and then select only a subset of acquired k-space lines for the image reconstruction. We will use a 2D Cartesian data set and hence we can select a different subset of $k_y$ points. 

Of course this approach has several limitations. We cannot got from a Cartesian to a non-Cartesian (e.g. radial) sampling pattern and we cannot adapt the overall FOV of the scan, but it is a start.

So let's think about what we need to do.

 * (A) We need a fully sampled Cartesian data set. Let's take `ptb_resolutionphantom_fully_ismrmrd.h5` which is in `exercises_data_path('MR', 'PTB_ACRPhantom_GRAPPA')`. We need to load the data and we will already call `preprocess_acquisition_data()` for this data.
 * (B) Then we need to find out, which $k_y$ points have been acquired and where the centre of k-space is (i.e. $k_y$ = 0), because for any undersampled MR acquisition it is a good idea to have a small fully sampled region around the k-space centre and carry out the undersampling in the higher k-space frequencies. 
 * (C) Define a subset of the orignal $k_y$ points.
 * (D) Create a new `AcquisitionData` object with only the subset defined in (C).
 * (E) Do a simple reconstruction to check we did the right thing.
 
After completing all the previous notebooks you are already MR reconstruction experts. Therefore, you should be able to do these steps by yourself. Give it a try and try to create a new k-space with an undersampling factor of 4 and a fully sampled central region of 10 $k_y$ points. Only have a look at the example solution below if you are stuck. A few hints to get you started:

 * Details on how to get information about the acquired k-space (e.g. which $k_y$ points have been acquired) can be find in the notebook `d_undersampled_reconstructions`.
 * We can define an empty `AcquisitionData` object using `acq_new = preprocessed_data.new_acquisition_data(empty=True)`.
 * We can select an `Acquisition` object from an existing `AcquisitionData` object using `cacq = existing_acq_data.acquisition(acq_index)`.
 * Finally we can add this `Acquisition` object to our so far empty `AcquisitionData` object using `acq_new.append_acquisition(cacq)`.
 * Once we have added all your `Acquisition` objects, we have to sort the data again: `acq_new.sort()`.
 * In order to find out, how to do a simple reconstruction have a look at the MR part of the introductory notebook `acquisition_model_mr_pet_ct`.

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib widget

In [ ]:
__version__ = '0.1.1'

import notebook_setup
import numpy

# import engine module
import sirf.Gadgetron as mr
from sirf_exercises import exercises_data_path

# import further modules
import os
import matplotlib.pyplot as plt


You should already have called the following script. If not, uncomment BOTH lines and run it now!

In [ ]:
#%%bash 
#bash ../../scripts/download_data.sh -m

# Utilities

In [ ]:
# First define some handy function definitions
# To make subsequent code cleaner, we have a few functions here. You can ignore
# ignore them when you first see this demo.

def plot_2d_image(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot to plot 2D image"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (numpy.array(vol.shape) - numpy.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    
    # Make a copy of the template to ensure we do not overwrite it
    templ_im_out = templ_im.copy()
    
    # Fill image content 
    templ_im_out.fill(numpy.reshape(vol, idim_orig))
    return(templ_im_out)

## (A) Fully sampled k-space data

Load in fully sampled k-space data and preprocess it.

In [ ]:
# Load MR AcquisitionData
mr_acq = mr.AcquisitionData(exercises_data_path('MR', 'PTB_ACRPhantom_GRAPPA') 
                            + '/ptb_resolutionphantom_fully_ismrmrd.h5' )
preprocessed_data = mr.preprocess_acquisition_data(mr_acq)

In [ ]:
# Calculate image
recon = mr.FullySampledReconstructor()
recon.set_input(preprocessed_data)
recon.process()
im_mr = recon.get_output()

In [ ]:
# Display it
plt.figure();
plot_2d_image([1,1,1], numpy.abs(im_mr.as_array())[0,:,:], 'Original image', cmap="Greys_r")

## (B) Find out which k-space points have been acquired

We will get the information about the $k_y$ position for each `Acquisition`. Because we have to go through all the acquired data, this can take a bit of time.

In [ ]:
ky_index = preprocessed_data.get_ISMRMRD_info('kspace_encode_step_1')
print(ky_index)

So we have got 256 phase encoding points $k_y$. Because this is a fully sampled Cartesian acquisition we can savely assume that the k-space centre is located bang in the middle, i.e. $k_y$ = 0 for `ky_index` = 128.

## (C) Define a subset of k-space data

Let's start with something easy. Define a subset for a regular undersampling factor R = 2 but with a fully sampled central k-space region of 20 $k_y$ points.

In [ ]:
# Define an undersampling factor
R = 4

# Define the number of fully sampled k-space points in the k-space centre
N_ctr = 10

# and your k-space centre was in the middle, so 
ky0_index = len(ky_index)//2

Let's first select the fully sampled k-space centre

In [ ]:
ky_index_subset =  numpy.arange(ky0_index-N_ctr//2, ky0_index+N_ctr//2)

Now we can add the rest of the data with an undersampling factor of R

In [ ]:
ky_index_subset = numpy.concatenate((ky_index_subset, numpy.arange(start=0, stop=len(ky_index), step=R)), axis=0)

Of course, now we might have added points from the centre of k-space again. To make sure that no $k_y$ index occurs twice, we simply call `numpy.unique`

In [ ]:
ky_index_subset = numpy.unique(ky_index_subset)

Now we can plot the original fully sampled and the new undersampled indices. Hint: zoom into the figure.

In [ ]:
plt.figure()
plt.plot(ky_index, numpy.ones(ky_index.shape), 'bo')
plt.plot(ky_index[ky_index_subset], numpy.ones(ky_index[ky_index_subset].shape), 'r.');

## (D) Create new acquisition data

Now we know which k-space points to select, we need to select them and create a new `AcquisitionData` object. We will follow the steps detailed at the beginning of the notebook.

In [ ]:
acq_new = preprocessed_data.new_acquisition_data(empty=True)

# Create raw data
for jnd in range(len(ky_index_subset)):
    cacq = preprocessed_data.acquisition(ky_index_subset[jnd])
    acq_new.append_acquisition(cacq)

acq_new.sort()     

## (E) Simple reconstruction

Now we will do a simple reconstruction by defining and `AcquisitionModel` based on the `AcquisitionData` and then call `backward()` (i.e. Fourier transform).

In [ ]:
# Original data
csm_orig = mr.CoilSensitivityData()
csm_orig.smoothness = 200
csm_orig.calculate(preprocessed_data)

A_orig = mr.AcquisitionModel(preprocessed_data, im_mr)
A_orig.set_coil_sensitivity_maps(csm_orig)

im_orig = A_orig.backward(preprocessed_data)


# Undersampled data
csm_new = mr.CoilSensitivityData()
csm_new.smoothness = 200
csm_new.calculate(acq_new)

A_new = mr.AcquisitionModel(acq_new, im_mr)
A_new.set_coil_sensitivity_maps(csm_orig)

im_new = A_orig.backward(acq_new)

In [ ]:
# Display it
plt.figure();
plot_2d_image([1,2,1], numpy.abs(im_orig.as_array())[0,:,:], 'Original image', cmap="Greys_r")
plot_2d_image([1,2,2], numpy.abs(im_new.as_array())[0,:,:], 'Undersampled image', cmap="Greys_r")

Here we used the coil sensitivity maps from the original data. This is of course not correct, but we should calculate the coil sensitivity maps again from our new `AcquisitionData`. Nevertheless, for a Cartesian sampling scheme, the coil sensitivity maps need to be calculated from the fully sampled k-space centre. For the algorithm to detect which $k_y$ points belong to this fully sampled k-space centre, these points need to be labelled. The MR scanner does this automatically but here we cannot do it.

Now you have got a framework to simulate different undersampling patterns. Now you can try

   * different undersmapling factors
   * different number of points for the fully sampled central k-space region
   * calculate the coil maps from the newly created data and compare them to the original data
   * random undersampling and compare it to the regular undersampling
   * ...

In the notebook `cil_joint_tv_mr` we will use what we learned here again, but the code above will be packaged into a function to make it easier to use.